# Problema do Caixeiro Viajante
### *The Travelling Salesman Problem-TSP* 
É um problema de otimização NP-difícil inspirado na necessidade dos vendedores em realizar entregas em diversos locais, percorrendo o menor caminho possível, reduzindo o tempo necessário para a viagem e os possíveis custos com transporte e combustível. 

O TSP pode ser assim formulado: existem n pontos e caminhos entre todos eles com
comprimentos conhecidos. O objetivo é encontrar o caminho mais curto, que passa
por todos os pontos uma única vez, começando e terminando no mesmo ponto. Isso
significa que o objetivo é encontrar a viagem de ida e volta mais curta possível.

**Neste relatório iremos comparar algoritmos aproximados e exatos para resolver as instâncias do caixeiro viajante**

O relatório em PDF dos algoritmos e testes está disponível em: [Link do Drive](https://drive.google.com/file/d/1vWd6E7mb5MYohAkFWJyNxMFABnJfYLjV/view?usp=sharing)

Para utilizar os testes no Colab é necessário criar uma pasta com o nome 'testes' com os arquivos disponibilizados no e-aula. Visto que, a cada vez que o projeto é reciclado os arquivos somem. Busquei uma integração com o meu Drive, mas os arquivos também sumiram após a reciclagem (atualização da página).

## Bibliotecas

In [ ]:
!pip install munkres

In [ ]:
import numpy as np
import itertools
import time
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse import csr_matrix
from itertools import combinations
from numpy import triu, inf, array
from numpy import loadtxt
from munkres import Munkres

## Algoritmo Exato
### Algoritmo Ingênuo para o TSP: Tentar todos os caminhos.
A ideia do algoritmo, como o próprio nome é autoexplicativo, é utilizarmos uma força bruta, testando todos os caminhos e escolhendo o menor.

**Complexidade: O(n!), onde n = |V|**

**Exemplo: Para N = 6, temos quantos caminhos possíveis?**

**5! = 5x4x3x2x1 = 120**

**Ele é ótimo? Sim. Ele encontra a melhor solução**

**Ele é eficiente? De jeito nenhum... Se o número de entradas for muito grande poderá exceder o tempo de execução** 



In [ ]:
def TSPBruteForce(arquivo):
  tempoInicial = time.monotonic()
  arq = "testes/" + str(arquivo)
  file = open(arq)

  lines = list()

  for line in file:
    lines.append(line.split())

  file.close()

  lines = np.array(lines)
  n = lines.shape[0]
  indice = np.arange(n)
  result = {'caminho': indice, 'custo': 0}
  custoTotal = result['custo']
  caminhoFinal = result['caminho'] 
  custo = 0

  for i in range(n-1):
    custo += int(lines[i][i+1])

  custo += int(lines[n-1][0])

  custoTotal = custo 
  permut = itertools.permutations(indice)

  for eventoPos in permut:
    custo = 0

    for i in range(n - 1):
      custo += int(lines[eventoPos[i]][eventoPos[i+1]])

    custo += int(lines[eventoPos[n-1]][eventoPos[0]])

    if custo < custoTotal:
      custoTotal = custo
      caminhoFinal = eventoPos

  tempoFinal = time.monotonic()
  tempoTotal = tempoFinal - tempoInicial
  return custoTotal, caminhoFinal, tempoTotal

In [ ]:
custo, caminho, tempoFinal = TSPBruteForce('tsp1_253.txt')
print(f'Algoritmo Exato (Força Bruta)'
      f'\n\tCaminho: {caminho}'
      f'\n\tCusto: {custo}'
      f'\n\tTempo de execução: {"{:.5f}".format(tempoFinal)} segundos')

Algoritmo Exato (Força Bruta)
	Caminho: (0, 7, 4, 3, 9, 5, 2, 6, 1, 10, 8)
	Custo: 253
	Tempo de execução: 396.95524 segundos


In [ ]:
custo, caminho, tempoFinal = TSPBruteForce('tsp2_1248.txt')
print(f'Algoritmo Exato (Força Bruta)'
      f'\n\tCaminho: {caminho}'
      f'\n\tCusto: {custo}'
      f'\n\tTempo de execução: {"{:.5f}".format(tempoFinal)} segundos')

IndexError: ignored

In [ ]:
custo, caminho, tempoFinal = TSPBruteForce('tsp3_1194.txt')
print(f'Algoritmo Exato (Força Bruta)'
      f'\n\tCaminho: {caminho}'
      f'\n\tCusto: {custo}'
      f'\n\tTempo de execução: {"{:.5f}".format(tempoFinal)} segundos')

In [ ]:
custo, caminho, tempoFinal = TSPBruteForce('tsp4_7013.txt')
print(f'Algoritmo Exato (Força Bruta)'
      f'\n\tCaminho: {caminho}'
      f'\n\tCusto: {custo}'
      f'\n\tTempo de execução: {"{:.5f}".format(tempoFinal)} segundos')

In [ ]:
custo, caminho, tempoFinal = TSPBruteForce('tsp5_27603.txt')
print(f'Algoritmo Exato (Força Bruta)'
      f'\n\tCaminho: {caminho}'
      f'\n\tCusto: {custo}'
      f'\n\tTempo de execução: {"{:.5f}".format(tempoFinal)} segundos')

### Dissertação sobre os testes do Algoritmo exato
A maioria dos testes não foi possível rodar em tempo plausível, apenas nos dois primeiros (tsp1\_253.txt e tsp2\_1248.txt) o nosso algoritmo de força bruta nos deu a solução esperada, a exata. O máximo de tempo que deixei rodando o algoritmo no Colab foram aproximadamente 60 minutos, contudo, como o tempo de execução difere entre máquinas, seria interessante testar em outra com mais capacidade de processamento. 

Como já era esperado, o nosso algoritmo ingênuo, depois de um longo tempo, para entradas específicas, encontrou a solução exata para o problema do Caixeiro Viajante. Partindo desse princípio, eu consideraria ele um bom algoritmo, pois encontrou a solução. Contudo, não consideraria um algoritmo eficiente, pois para entradas maiores ele pode exceder o tempo de execução.

## Algoritmo Aproximativo
Existem heurísticas para resolver o problema do caixeiro viajante, que são métodos por meio dos quais a solução ótima para o TSP é procurada. Embora as heurísticas não possam
garantir que a solução ótima seja encontrada, ou não ao menos em tempo real, um
grande número delas encontrará uma solução próxima à ótima, ou mesmo
encontrará uma solução ótima para certos casos do problema do caixeiro viajante.

Utilizaremos uma heurística construtiva para a solução do TSP utilizando um algoritmo de Christofides.

Informações detalhadas sobre o algoritmo e os testes dado neste Notebook estão disponíveis no relatório.

    ##PASSOS
    # Criar uma árvore de abrangência minima
    # Criar uma representação gráfica usando o dicionário python
    # Obtér vértices de grau ímpar
    # Gerar uma combinação entre todos os vértices de grau ímpar
    # Usar o algoritmo de Munkres para encontrar uma correspondência perfeita de peso mínimo
    # Formar um circuito euleriano
    # Converter para um circuito hamiltoniano (atalho)
    # Obter custódia total do circuito hamiltoniano

In [ ]:
import os
import re

N_VERTICES = None
DIR = 'testes/'
def run():
    
    for filename in os.listdir(DIR):
        with open(f'{DIR}{filename}', 'r') as file:
            lines = file.readlines()

        lines = [re.sub('\s+', ',', line.strip()) for line in lines]

        with open(f'{DIR}{filename}', 'w') as file:
            file.writelines(line + '\n' for line in lines)

    matrizAdjacencia = loadtxt(f'{DIR}tsp4_7013.txt', dtype='int', delimiter=',')
    matrizAdjacencia_fixed = matrizAdjacencia[:22, :22]
    lines = [re.sub('\s+', ',', " ".join(map(str, line))) for line in matrizAdjacencia_fixed]
    with open(f'{DIR}tsp4_7013_fixed.txt', 'w+') as file:
        file.writelines(line + '\n' for line in lines)
    

def christofides(arquivo):
    tempoInicial = time.monotonic()
    run()
    matrizAdjacencia = loadtxt(f'{DIR}{arquivo}.txt', dtype='int', delimiter=',')
    global N_VERTICES
    N_VERTICES = len(matrizAdjacencia)

    
    mst = minimum_spanning_tree(csr_matrix(triu(matrizAdjacencia)))
    grafo = criaGrafo(matrizAdjacencia, mst)  
    verticesImpar = VerticeGrauImpar(grafo) 
    combVertImp = [set(i) for i in combinations(set(verticesImpar), len(verticesImpar) // 2)]
    vertImpSubgrafo = verticesImpar_subgrafo(matrizAdjacencia, combVertImp, verticesImpar)
    munkres(matrizAdjacencia, grafo, vertImpSubgrafo)
    caminho = caminhoEuleriano(grafo)
    menorCaminho = caminhoHamiltoniano(caminho)
    custo = caminho_custo(matrizAdjacencia, menorCaminho)


    tempoFinal = time.monotonic()
    tempoTotal = tempoFinal - tempoInicial
    return custo, menorCaminho, tempoTotal


def criaGrafo(matrizAdjacencia, mst):
    grafo = {}

    for v1, v2 in list(zip(*mst.nonzero())):

        if v1.item() not in grafo:
            grafo[v1.item()] = {}
        if v2.item() not in grafo:
            grafo[v2.item()] = {}

        grafo[v1.item()][v2.item()] = matrizAdjacencia[v1.item()][v2.item()]
        grafo[v2.item()][v1.item()] = matrizAdjacencia[v1.item()][v2.item()]

    return grafo


def VerticeGrauImpar(mst):
    verticesImpar = [vertex for vertex, neighbors in mst.items() if len(neighbors) % 2 == 1]

    return sorted(verticesImpar)


def verticesImpar_subgrafo(matrizAdjacencia, combVertImp, verticesImpar):
    subgrafos = []
    vertex_sets = []
    for vertex_set1 in combVertImp:
        vertex_set1 = list(sorted(vertex_set1))
        vertex_set2 = []
        for vertex in verticesImpar:
            if vertex not in vertex_set1:
                vertex_set2.append(vertex)
        matriz = [[inf for _ in enumerate(vertex_set2)] for _ in enumerate(vertex_set1)]

        for i in range(len(vertex_set1)):
            for j in range(len(vertex_set2)):
                matriz[i][j] = matrizAdjacencia[vertex_set1[i]][vertex_set2[j]]

        subgrafos.append(matriz)

        vertex_sets.append([vertex_set1, vertex_set2])

    return [subgrafos, vertex_sets]


def munkres(matrizAdjacencia, grafo, vertImpSubgrafo):
    m = Munkres()
    minimo = inf

    for indice, bip_grafo in enumerate(vertImpSubgrafo[0]):
        munkres_indicees = m.compute(bip_grafo)

        custo = 0
        for idx in munkres_indicees:
            custo += bip_grafo[idx[0]][idx[1]]

        if custo < minimo:
            minimo = custo
            min_indice = indice
            min_munkres_indicees = munkres_indicees

    munkres_indicees = [[]] * len(min_munkres_indicees)

    for indice, vertex_set in enumerate(min_munkres_indicees):
        munkres_indicees[indice].append(vertImpSubgrafo[1][min_indice][0][vertex_set[0]])
        munkres_indicees[indice].append(vertImpSubgrafo[1][min_indice][1][vertex_set[1]])

    for match in munkres_indicees:
        grafo[match[0]][match[1]] = matrizAdjacencia[match[0]][match[1]]
        grafo[match[1]][match[0]] = matrizAdjacencia[match[0]][match[1]]


def caminhoEuleriano(grafo):
    iniVertex = list(grafo[0])[0]
    caminho = [list(grafo[iniVertex])[0]]

    while len(grafo) > 0:

        for indice, vertex in enumerate(caminho):
            if vertex in grafo and len(grafo[vertex]) > 0:
                break

        while vertex in grafo:
            prox_vertex = list(grafo[vertex])[0]

            del grafo[vertex][prox_vertex]
            del grafo[prox_vertex][vertex]

            if len(grafo[vertex]) == 0:
                del grafo[vertex]
            if len(grafo[prox_vertex]) == 0:
                del grafo[prox_vertex]

            indice += 1
            caminho.insert(indice, prox_vertex)
            vertex = prox_vertex

    return caminho


def caminhoHamiltoniano(caminho):
    menorCaminho = []
    for vertex in caminho:
        if vertex not in menorCaminho:
            menorCaminho.append(vertex)

    menorCaminho = list(array(menorCaminho) + 1)
    menorCaminho.append(1)

    return menorCaminho


def caminho_custo(matrizAdjacencia, caminho):
    peso = 0
    for i in range(len(caminho) - 1):
        peso += matrizAdjacencia[caminho[i]-1, caminho[i + 1]-1]

    return peso


In [ ]:
custo, caminho, tempoFinal = christofides('tsp1_253')
print(f'Christofides (Algoritmo Aproximativo)'
      f'\n\tCaminho: {caminho}'
      f'\n\tCusto: {custo}'
      f'\n\tTempo de execução: {"{:.5f}".format(tempoFinal)} segundos')

Christofides (Algoritmo Aproximativo)
	Caminho: [1, 9, 8, 3, 7, 2, 11, 5, 4, 6, 10, 1]
	Custo: 265
	Tempo de execução: 0.01807 segundos


In [ ]:
custo, caminho, tempoFinal = christofides('tsp2_1248')
print(f'Christofides (Algoritmo Aproximativo)'
      f'\n\tCaminho: {caminho}'
      f'\n\tCusto: {custo}'
      f'\n\tTempo de execução: {"{:.5f}".format(tempoFinal)} segundos')

Christofides (Algoritmo Aproximativo)
	Caminho: [1, 2, 6, 5, 4, 3, 1]
	Custo: 1272
	Tempo de execução: 0.01164 segundos


In [ ]:
custo, caminho, tempoFinal = christofides('tsp3_1194')
print(f'Christofides (Algoritmo Aproximativo)'
      f'\n\tCaminho: {caminho}'
      f'\n\tCusto: {custo}'
      f'\n\tTempo de execução: {"{:.5f}".format(tempoFinal)} segundos')

Christofides (Algoritmo Aproximativo)
	Caminho: [1, 2, 10, 11, 12, 13, 14, 15, 9, 8, 6, 7, 5, 4, 3, 1]
	Custo: 1360
	Tempo de execução: 0.01510 segundos


In [ ]:
custo, caminho, tempoFinal = christofides('tsp4_7013')
print(f'Christofides (Algoritmo Aproximativo)'
      f'\n\tCaminho: {caminho}'
      f'\n\tCusto: {custo}'
      f'\n\tTempo de execução: {"{:.5f}".format(tempoFinal)} segundos')

In [ ]:
custo, caminho, tempoFinal = christofides('tsp5_27603')
print(f'Christofides (Algoritmo Aproximativo)'
      f'\n\tCaminho: {caminho}'
      f'\n\tCusto: {custo}'
      f'\n\tTempo de execução: {"{:.5f}".format(tempoFinal)} segundos')

Christofides (Algoritmo Aproximativo)
	Caminho: [1, 11, 10, 12, 13, 14, 17, 18, 22, 23, 29, 28, 26, 20, 16, 25, 27, 24, 21, 19, 15, 8, 4, 2, 6, 5, 7, 9, 3, 1]
	Custo: 38110
	Tempo de execução: 0.52984 segundos


### Dissertação sobre os testes do Algoritmo Aproximativo

Em todos os testes do nosso algoritmo aproximativo, a solução dada se aproxima bastante da ótima, e diferente do último método visto, feito por uma solução força bruta, onde o nosso algoritmo nos dava uma solução exata, o tempo de execução é imensamente inferior, o que nos dá uma vantagem gigantesca. Também deu pra notar que com entradas muito maiores o custo já difere um pouco da solução ótima, o que também era esperado, a exemplo disso temos o teste 5, onde a solução ótima seria dada por um custo de 27603 e o nosso algoritmo aproximativo nos deu um custo de 38110.

A única exceção nos nossos testes foi o teste 4 "tsp4_7013", onde o colab nos reportou um erro de uso total da capacidade da RAM. Acredito que essa exceção nos dá por causa da versão gratuita do colab, e para fins de comparação seria interessante rodar em uma máquina ou SO, ao invés de rodar pelo colab.

# Conclusão

Considerei um problema muito interessante, já tinha tido um pré-conhecimento sobre o problema de otimização do TSP, e particularmente achei um trabalho muito estimulante de ser desenvolvido, visto que, o desenvolvimento de algoritmos de aproximação é uma das linhas de pesquisa que mais têm crescido na área de otimização combinatória e teoria da computação.

Notamos que para o problema do TSP em dimensões elevadas, a resolução por métodos de força bruta é proibitiva em termos de tempos computacionais. Portanto, a abordagem de solução heurística e de aproximação é mais útil para as aplicações que preferem o tempo de execução do algoritmo em relação à precisão do resultado.

Para a grande maioria dos problemas de otimização interessantes não se conhecem algoritmos "exatos" eficientes, este também é o caso do problema do Caixeiro Viajante. Por isso para esse problema em específico os algoritmos de aproximação são muito úteis. 

Conclui também que, o nosso algoritmo aproximativo (Christofides), nos dá uma eficiência maior em termos de tempo computacional e a precisão do resultado não difere tanto em pequenas e médias dimensões, por isso escolheria esse método de heurística se estivesse desenvolvendo uma solução para o problema, ao invés da ótima.